In [ ]:
!pip install transformers tensorflow-addons focal_loss

In [ ]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
import re
import io
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
from zipfile import ZipFile
from IPython.display import FileLink 
import tensorflow as tf

np.random.seed(45)
tf.random.set_seed(45)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tf.distribute.cluster_resolver.TPUClusterResolver.connect())

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.7.243.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.7.243.218:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
train_data=pd.read_csv("/content/train.csv")
valid_data=pd.read_csv("/content/dev.csv")

In [ ]:
# OFF=train_data.loc[(train_data["OFF_label"]==1)&(train_data["HS_label"]==0)].sample(1100,)
# NORM=train_data.loc[train_data["OFF_label"]==0].sample(1100)
# HS=train_data.loc[train_data["HS_label"]!=0]
# train_data=OFF.append(NORM)
# train_data=train_data.append(HS)
# # train_data=train_data.append(HS)
# # train_data=train_data.append(HS)
# train_data=train_data.sample(frac=1).reset_index(drop=True)

In [ ]:
def encode_labels(data):
    encoded_OFF=tf.convert_to_tensor(data["OFF_label"].astype("int32"))
    #encoded_HS=tf.keras.utils.to_categorical(data["HS_label"],num_classes=7)
    encoded_HS=tf.convert_to_tensor(data["HS_label"].astype("int32"))
    text=list(data["tweet_text"].astype("str"))
    return text,encoded_OFF,encoded_HS

train_text,train_OFF,train_HS=encode_labels(train_data)
valid_text,valid_OFF,valid_HS=encode_labels(valid_data)

In [ ]:
def OFF_not_HS_feature_gen(encoded_OFF,encoded_HS):
    OFF_not_HS=[]
    for x,y in zip(encoded_OFF,encoded_HS):
        OFF_not_HS.append(int(x==1 and y==0))
    return tf.convert_to_tensor(OFF_not_HS)

def Binary_HS_feature_gen(encoded_HS):
    HS_bn=[]
    for x in encoded_HS:
        HS_bn.append(x!=0)
    return tf.convert_to_tensor(HS_bn)

train_OFF_not_HS=OFF_not_HS_feature_gen(train_OFF,train_data["HS_label"])
train_HS_bn=Binary_HS_feature_gen(train_data["HS_label"])
valid_OFF_not_HS=OFF_not_HS_feature_gen(valid_OFF,valid_data["HS_label"])
valid_HS_bn=Binary_HS_feature_gen(valid_data["HS_label"])

In [ ]:
def tokenize(sentences):
    tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERTv2")
    input_ids, input_masks = [],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=256,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
    return np.asarray(input_ids, dtype='int32'),np.asarray(input_masks, dtype='int32')

train_input_ids,train_input_masks=tokenize(train_text)
valid_input_ids,valid_input_masks=tokenize(valid_text)

In [ ]:
def report_gen(predictions,labels):
    report={
    "F1_macro":f1_score(predictions,labels,average="macro"),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels,average="macro"),
    "Recall_macro":recall_score(predictions,labels,average="macro")
    }
    return report

def eval_taskA(predictions_OFF,labels_OFF,return_predictions=False):
    predictions_OFF=[int(i>0.95) for i in predictions_OFF]
    return predictions_OFF if return_predictions else report_gen(predictions_OFF,labels_OFF)

def eval_taskB(predictions_HS,labels_HS_bn,return_predictions=False):
    predictions_HS_bn=[]
    for pred in predictions_HS:
        pred=list(pred)
        max_value = max(pred)
        max_index = pred.index(max_value)
        predictions_HS_bn.append(max_index!=0)
    return predictions_HS_bn if return_predictions else report_gen(predictions_HS_bn,labels_HS_bn)


def eval_taskC(predictions_HS,labels_HS,return_predictions=False):
    predictions_HS=tf.argmax(predictions_HS,axis=1)
    return predictions_HS if return_predictions else report_gen(predictions_HS,labels_HS)

def eval_taskD(predictions_OFF_not_HS,labels_OFF_not_HS):
    predictions_OFF_not_HS=[int(i>0.95) for i in predictions_OFF_not_HS]
    return report_gen(predictions_OFF_not_HS,labels_OFF_not_HS)


In [ ]:
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.layers import Layer, InputSpec
from keras.utils.conv_utils import conv_output_length

def _dropout(x, level, noise_shape=None, seed=None):
    x = K.dropout(x, level, noise_shape, seed)
    x *= (1. - level) # compensate for the scaling by the dropout
    return x


class QRNN(Layer):
    '''Quasi RNN
    # Arguments
        units: dimension of the internal projections and the final output.
    # References
        - [Quasi-recurrent Neural Networks](http://arxiv.org/abs/1611.01576)
    '''
    def __init__(self, units, window_size=2, stride=1,
                 return_sequences=False, go_backwards=False, 
                 stateful=False, unroll=False, activation='tanh',
                 kernel_initializer='uniform', bias_initializer='zero',
                 kernel_regularizer=None, bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None, bias_constraint=None, 
                 dropout=0, use_bias=True, input_dim=None, input_length=None,
                 **kwargs):
        self.return_sequences = return_sequences
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.units = units 
        self.window_size = window_size
        self.strides = (stride, 1)

        self.use_bias = use_bias
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = dropout
        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.input_dim = input_dim
        self.input_length = input_length
        if self.input_dim:
            kwargs['input_shape'] = (self.input_length, self.input_dim)
        super(QRNN, self).__init__(**kwargs)

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        self.input_dim = input_shape[2]
        self.input_spec = InputSpec(shape=(batch_size, None, self.input_dim))
        self.state_spec = InputSpec(shape=(batch_size, self.units))

        self.states = [None]
        if self.stateful:
            self.reset_states()

        kernel_shape = (self.window_size, 1, self.input_dim, self.units * 3)
        self.kernel = self.add_weight(name='kernel',
                                      shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(name='bias', 
                                        shape=(self.units * 3,),
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)

        self.built = True

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        length = input_shape[1]
        if length:
            length = conv_output_length(length + self.window_size - 1,
                                        self.window_size, 'valid',
                                        self.strides[0])
        if self.return_sequences:
            return (input_shape[0], length, self.units)
        else:
            return (input_shape[0], self.units)

    def compute_mask(self, inputs, mask):
        if self.return_sequences:
            return mask
        else:
            return None

    def get_initial_states(self, inputs):
        # build an all-zero tensor of shape (samples, units)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        initial_state = K.tile(initial_state, [1, self.units])  # (samples, units)
        initial_states = [initial_state for _ in range(len(self.states))]
        return initial_states

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        if not self.input_spec:
            raise RuntimeError('Layer has never been called '
                               'and thus has no states.')

        batch_size = self.input_spec.shape[0]
        if not batch_size:
            raise ValueError('If a QRNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')

        if self.states[0] is None:
            self.states = [K.zeros((batch_size, self.units))
                           for _ in self.states]
        elif states is None:
            for state in self.states:
                K.set_value(state, np.zeros((batch_size, self.units)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                 'but it received ' + str(len(states)) +
                                 'state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if value.shape != (batch_size, self.units):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, self.units)) +
                                     ', found shape=' + str(value.shape))
                K.set_value(state, value)

    def __call__(self, inputs, initial_state=None, **kwargs):
        # If `initial_state` is specified,
        # and if it a Keras tensor,
        # then add it to the inputs and temporarily
        # modify the input spec to include the state.
        if initial_state is not None:
            if hasattr(initial_state, '_keras_history'):
                # Compute the full input spec, including state
                input_spec = self.input_spec
                state_spec = self.state_spec
                if not isinstance(state_spec, list):
                    state_spec = [state_spec]
                self.input_spec = [input_spec] + state_spec

                # Compute the full inputs, including state
                if not isinstance(initial_state, (list, tuple)):
                    initial_state = [initial_state]
                inputs = [inputs] + list(initial_state)

                # Perform the call
                output = super(QRNN, self).__call__(inputs, **kwargs)

                # Restore original input spec
                self.input_spec = input_spec
                return output
            else:
                kwargs['initial_state'] = initial_state
        return super(QRNN, self).__call__(inputs, **kwargs)

    def call(self, inputs, mask=None, initial_state=None, training=None):
        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            initial_states = inputs[1:]
            inputs = inputs[0]
        elif initial_state is not None:
            pass
        elif self.stateful:
            initial_states = self.states
        else:
            initial_states = self.get_initial_states(inputs)

        if len(initial_states) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_states)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        if self.unroll and input_shape[1] is None:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')
        constants = self.get_constants(inputs, training=None)
        preprocessed_input = self.preprocess_input(inputs, training=None)

        last_output, outputs, states = K.rnn(self.step, preprocessed_input,
                                            initial_states,
                                            go_backwards=self.go_backwards,
                                            mask=mask,
                                            constants=constants,
                                            unroll=self.unroll,
                                            input_length=input_shape[1])
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        # Properly set learning phase
        if 0 < self.dropout < 1:
            last_output._uses_learning_phase = True
            outputs._uses_learning_phase = True

        if self.return_sequences:
            return outputs
        else:
            return last_output

    def preprocess_input(self, inputs, training=None):
        if self.window_size > 1:
            inputs = K.temporal_padding(inputs, (self.window_size-1, 0))
        inputs = K.expand_dims(inputs, 2)  # add a dummy dimension

        output = K.conv2d(inputs, self.kernel, strides=self.strides,
                          padding='valid',
                          data_format='channels_last')
        output = K.squeeze(output, 2)  # remove the dummy dimension
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')

        if self.dropout is not None and 0. < self.dropout < 1.:
            z = output[:, :, :self.units]
            f = output[:, :, self.units:2 * self.units]
            o = output[:, :, 2 * self.units:]
            f = K.in_train_phase(1 - _dropout(1 - f, self.dropout), f, training=training)
            return K.concatenate([z, f, o], -1)
        else:
            return output

    def step(self, inputs, states):
        prev_output = states[0]

        z = inputs[:, :self.units]
        f = inputs[:, self.units:2 * self.units]
        o = inputs[:, 2 * self.units:]

        z = self.activation(z)
        f = f if self.dropout is not None and 0. < self.dropout < 1. else K.sigmoid(f)
        o = K.sigmoid(o)

        output = f * prev_output + (1 - f) * z
        output = o * output

        return output, [output]

    def get_constants(self, inputs, training=None):
        return []
 
    def get_config(self):
        config = {'units': self.units,
                  'window_size': self.window_size,
                  'stride': self.strides[0],
                  'return_sequences': self.return_sequences,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll,
                  'use_bias': self.use_bias,
                  'dropout': self.dropout,
                  'activation': activations.serialize(self.activation),
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'input_dim': self.input_dim,
                  'input_length': self.input_length}
        base_config = super(QRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def create_model(transformer,conv_units=128,qrnn_units=256,dense_units=64):
    input_ids= tf.keras.layers.Input(shape=(256,), dtype='int32')
    input_masks = tf.keras.layers.Input(shape=(256,), dtype='int32')
    embedding_layer=transformer(input_ids, attention_mask=input_masks)[0]

    OFF = tf.keras.layers.Conv1D(conv_units, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(embedding_layer)
    OFF_f = QRNN(qrnn_units)(OFF)  
    OFF_b = QRNN(qrnn_units, go_backwards=True)(OFF)
    OFF = tf.keras.layers.concatenate([OFF_f, OFF_b])
    OFF =tf.keras.layers.Dropout(0.3,seed=52)(OFF)
    output_OFF=tf.keras.layers.Dense(1, activation="sigmoid",name="output_OFF")(OFF)
    
    HS = tf.keras.layers.Conv1D(conv_units, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(embedding_layer)
    HS_f = QRNN(qrnn_units)(HS)  
    HS_b = QRNN(qrnn_units, go_backwards=True)(HS)
    HS = tf.keras.layers.concatenate([HS_f, HS_b])
    HS =tf.keras.layers.Dropout(0.3,seed=52)(HS)    
    output_HS=tf.keras.layers.Dense(7, activation="softmax",name="output_HS")(HS)

    model = tf.keras.Model(inputs=[input_ids, input_masks], outputs = [output_OFF,output_HS])
    
    return model

In [ ]:
def build_model():
    with tpu_strategy.scope():
        config =AutoConfig.from_pretrained("UBC-NLP/MARBERTv2",dropout=0.3,seed=3,attention_dropout=0.3,output_hidden_states = True)
        transformer= TFAutoModel.from_pretrained("UBC-NLP/MARBERTv2",config=config)
        return create_model(transformer)

model=build_model()

def train_model(model,input_ids,input_masks,label_OFF,label_HS,batch_size=32,epochs_frozen=4,epochs_unfrozen=16,verbose=1):
   
    if epochs_frozen:
        
        for layer in model.layers[:3]:
            layer.trainable = False

        with tpu_strategy.scope():
            model.compile(loss={"output_OFF":"binary_crossentropy","output_HS":"sparse_categorical_crossentropy"},
                          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                          metrics={"output_OFF":"accuracy","output_HS":"sparse_categorical_accuracy"}
                          #loss_weights={"output_OFF":1,"output_HS":1.5}
                         )

            model.fit([input_ids, input_masks],[label_OFF,label_HS],batch_size=64,epochs=epochs_frozen,verbose=verbose,
                   validation_data=([valid_input_ids,valid_input_masks], [valid_OFF,valid_HS]))
        
    if epochs_unfrozen:
        
        for layer in model.layers[:3]:
            layer.trainable = True

        with tpu_strategy.scope():
            model.compile(loss={"output_OFF":"binary_crossentropy","output_HS":"sparse_categorical_crossentropy"},
                          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                          metrics={"output_OFF":"accuracy","output_HS":"sparse_categorical_accuracy"}
                          #loss_weights={"output_OFF":1,"output_HS":1.5}
                         )

            model.fit([input_ids, input_masks],[label_OFF,label_HS],batch_size=64,epochs=epochs_unfrozen,verbose=verbose,
                    validation_data=([valid_input_ids,valid_input_masks], [valid_OFF,valid_HS]))

train_model(model,train_input_ids,train_input_masks,train_OFF,train_HS)

In [ ]:
def evaluate_model(model,input_ids,input_masks,labels_OFF,labels_HS,labels_OFF_not_HS,labels_HS_bn):
    predictions_OFF,predictions_HS=model.predict([input_ids,input_masks])
    print(f"TaskA:{eval_taskA(predictions_OFF,labels_OFF)}")
    print(f"TaskB:{eval_taskB(predictions_HS,labels_HS_bn)}")
    print(f"TaskC:{eval_taskC(predictions_HS,labels_HS)}")
evaluate_model(model,valid_input_ids,valid_input_masks,valid_OFF,valid_HS,valid_OFF_not_HS,valid_HS_bn)

In [ ]:
evaluate_model(model,train_input_ids,train_input_masks,train_OFF,train_HS,train_OFF_not_HS,train_HS_bn)

In [ ]:
def zip_file(task):
    with ZipFile(task+".zip", 'w') as myzip:
        myzip.write(task+".txt")
    myzip.close()
    
def submission_gen_taskA(predictions):
    sub=[]
    for i in predictions:
        if i==1:
            sub.append("OFF")
        else :
            sub.append("NOT_OFF")
    pd.DataFrame(sub).to_csv("TaskA.txt",index=False,header=False)

def submission_gen_taskB(predictions):
    sub=[]
    for i in predictions:
        if i==1:
            sub.append("HS")
        else :
            sub.append("NOT_HS")
    pd.DataFrame(sub).to_csv("LB.txt",index=False,header=False)

def submission_gen_taskC(predictions):
    sub=[]
    for i in predictions:
        if i==0:
            sub.append("NOT_HS")
        else:
            sub.append("HS"+str(i))
    pd.DataFrame(sub).to_csv("LC.txt",index=False,header=False)
    
def submission_gen():
    submission_gen_taskA(eval_taskA(ta,valid_OFF,return_predictions=True))
    submission_gen_taskB(eval_taskB2(tc,valid_HS,return_predictions=True))
    submission_gen_taskC(eval_taskC(valid_OFF,valid_HS_bn,return_predictions=True))

#submission_gen()